# Segmenting and Clusering Neighbourhoods in Toronto

<b>Importing Lib to get data in requred formate</b>

In [1]:
import requests as rq
web_url = rq.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [2]:
from bs4 import BeautifulSoup
Soup = BeautifulSoup(web_url,'lxml')

<b>By observation we can see that the tabular data is availabe in table and belongs to class="wikitable sortable"So let's extract only table</b>

In [3]:
my_table = Soup.find('table',{'class','wikitable sortable'})
print(my_table.tr.text)


Postcode
Borough
Neighbourhood



<b> Getting all datas in tr and seperating each td within by "," and write csv file from result </b>

In [4]:
headers = "Postcode,Borough,Neighbourhood"
table1=""
for tr in my_table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1 = table1+row1[1:]        
file = open("toronto.csv","wb")
file.write(bytes(table1,encoding="ascii",errors="ignore"))

8738

<b> Converting into dataframe and assinging columnnames </b>

In [5]:
import pandas as pd
df = pd.read_csv('toronto.csv',header= None)
df.columns=["Postalcode","Borough","Neighbourhood"]
df.head(10)

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


<b> Only processing the cells that have an assigned borough. Ignoring the cells with a borough that is Not assigned. Droping row where borough is "Not assigned" </b>

In [6]:
indexNames = df[df['Borough']=='Not assigned'].index
df.drop(indexNames, inplace=True)
df.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


<b> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough </b>

In [7]:
df.loc[df['Neighbourhood']== 'Not assigned','Neighbourhood']= df['Borough']
df.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


<b> rows will be same postalcode will combined into one row with the neighborhoods separated with a comma </b>

In [8]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg(','.join)
df_new = result.reset_index()
df_new.head(13)

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [9]:
df_new.shape

(103, 3)